In [1]:
import gc
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
pd.options.display.float_format = "{:,.4f}".format
import random
import sys
sys.path.append("../../")
import time
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
from pandarallel import pandarallel
from tqdm import tqdm

In [2]:
from pathlib import Path
rootpath = Path.cwd().parent
sys.path.append(os.path.join(rootpath))

In [3]:
from utils.constants import *
from utils.eda_helpers import (
    plot_missing_proportion_barchart, get_cols, plot_scatterplot, plot_target_check, 
    plot_int_feature_distribution, plot_train_test_distribution, check_overlap_missing,
    insert_row_number, plot_sampled_time_series
)
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)
from utils.preprocess_helpers import *

In [4]:
%load_ext autoreload
%autoreload

In [5]:
gc.collect()

36

In [6]:
START = time.time()

In [7]:
NON_FEATURE_COLUMNS

['customer_ID', 'target', 'row_number', 'row_number_inv', 'S_2']

### Combine Train Data

In [8]:
%%time
train_sp = read_file(f"./train_parquet/train_spend_payment.parquet")
train_r = read_file(f"./train_parquet/train_risk.parquet")
train_b = read_file(f"./train_parquet/train_balance.parquet")
train_d1 = read_file(f"./train_parquet/train_delinquency_part1.parquet")
train_d2 = read_file(f"./train_parquet/train_delinquency_part2.parquet")

Shape of data: (5531451, 24)
Shape of data: (5531451, 26)
Shape of data: (5531451, 39)
Shape of data: (5531451, 49)
Shape of data: (5531451, 41)
CPU times: user 14.7 s, sys: 6.79 s, total: 21.5 s
Wall time: 7.05 s


In [9]:
%%time
train = pd.concat([
    train_sp.set_index(NON_FEATURE_COLUMNS),
    train_r.set_index(NON_FEATURE_COLUMNS),
    train_b.set_index(NON_FEATURE_COLUMNS),
    train_d1.set_index(NON_FEATURE_COLUMNS),
    train_d2.set_index(NON_FEATURE_COLUMNS)
], axis=1).reset_index()

CPU times: user 18.6 s, sys: 4.31 s, total: 23 s
Wall time: 22.9 s


In [10]:
train.shape

(5531451, 159)

In [11]:
train.columns

Index(['customer_ID', 'target', 'row_number', 'row_number_inv', 'S_2', 'P_2',
       'S_3', 'P_3', 'S_5', 'S_6',
       ...
       'D_133', 'D_134', 'D_136', 'D_141', 'D_142', 'D_144', 'D_145', 'D_183',
       'D_1314', 'D_1343'],
      dtype='object', length=159)

In [12]:
%%time
train.to_parquet(f"./train_parquet/train_all_variables.parquet")

CPU times: user 23.5 s, sys: 1.08 s, total: 24.6 s
Wall time: 20.7 s


In [13]:
train.head()

,customer_ID,target,row_number,row_number_inv,S_2,P_2,S_3,P_3,S_5,S_6,...,D_133,D_134,D_136,D_141,D_142,D_144,D_145,D_183,D_1314,D_1343
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,13,1,2017-03-09,0.9300,0.1200,0.7300,0.0200,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,12,2,2017-04-07,0.9300,0.1200,0.7200,0.0300,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,11,3,2017-05-28,0.9500,0.1200,0.7300,0.0400,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,10,4,2017-06-13,0.9600,0.1100,0.7400,0.0300,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,9,5,2017-07-16,0.9400,0.1100,0.6900,0.0500,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0


In [14]:
del train, train_sp, train_r, train_b, train_d1, train_d2

### Combine Test Data

In [15]:
%%time
test_sp = read_file(f"./test_parquet/test_spend_payment.parquet")
test_r = read_file(f"./test_parquet/test_risk.parquet")
test_b = read_file(f"./test_parquet/test_balance.parquet")
test_d1 = read_file(f"./test_parquet/test_delinquency_part1.parquet")
test_d2 = read_file(f"./test_parquet/test_delinquency_part2.parquet")

Shape of data: (11363762, 23)
Shape of data: (11363762, 25)
Shape of data: (11363762, 37)
Shape of data: (11363762, 48)
Shape of data: (11363762, 40)
CPU times: user 27.2 s, sys: 12.9 s, total: 40.1 s
Wall time: 13.4 s


In [16]:
cols = [c for c in NON_FEATURE_COLUMNS if c not in "target"]

In [17]:
insert_row_number(test_r)

Done insertion


In [18]:
%%time
test = pd.concat([
    test_sp.set_index(cols),
    test_r.set_index(cols),
    test_b.set_index(cols),
    test_d1.set_index(cols),
    test_d2.set_index(cols)
], axis=1).reset_index()

CPU times: user 36 s, sys: 7.8 s, total: 43.8 s
Wall time: 43.8 s


In [19]:
test.shape

(11363762, 157)

In [20]:
# cid_list = test["customer_ID"].unique().tolist()
# len(cid_list)
# split_index = 462311

In [21]:
test.to_parquet(f"./test_parquet/test_all_variables.parquet")
# test.loc[test["customer_ID"].isin(cid_list[split_index:])].to_parquet(f"./test_parquet/test_all_variables2_.parquet")

In [22]:
test.head()

,customer_ID,row_number,row_number_inv,S_2,P_2,S_3,P_3,S_5,S_6,S_7,...,D_133,D_134,D_136,D_141,D_142,D_144,D_145,D_183,D_1314,D_1343
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,9,1,2019-02-19,0.6300,0.1600,NaN,0.0000,1,0.1000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,2
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,8,2,2019-03-25,0.5800,0.2400,NaN,0.0000,1,0.1600,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,7,3,2019-04-25,0.6000,0.2600,NaN,0.0000,0,0.2100,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,6,4,2019-05-20,0.6100,0.1800,NaN,0.0000,0,0.1900,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,5,5,2019-06-15,0.5900,0.1800,0.5900,0.0000,0,0.2000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,0


In [23]:
# del test, test_sp, test_r, test_b, test_d1, test_d2